In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import seaborn as sns

# Load data

In [ ]:
data_dir = "../data/raw/"
group1_name = "st01253_by4742swain"

In [ ]:
filepath1 = data_dir + group1_name
timeseries1_filepath = filepath1 + "_timeseries.csv"
labels1_filepath = filepath1 + "_labels.csv"

timeseries_df = pd.read_csv(timeseries1_filepath, index_col=[0,1,2])
labels_df = pd.read_csv(labels1_filepath, index_col=[0,1,2])

In [ ]:
timeseries_df

In [ ]:
timeseries_dropna = timeseries_df.dropna()

In [ ]:
labels_df = labels_df == 1

# Compute periodogram

In [ ]:
from postprocessor.core.processes.fft import fft

In [ ]:
freqs, power = fft.as_function(timeseries_df)
freqs_dropna = freqs.dropna()
power_dropna = power.dropna()

In [ ]:
power_dropna

# Score/rank

In [ ]:
# Scores for each time series is simply the max height of power
scores = power_dropna.max(axis=1)
scores_sorted = scores.sort_values(ascending=False)

In [ ]:
max(scores_sorted)

# Plot one time series

In [ ]:
idx = 0
timeseries = timeseries_df.loc[scores_sorted.index[idx]]

fig, ax = plt.subplots(figsize=(10,3))
ax.plot(timeseries)
ax.xaxis.set_major_locator(ticker.MultipleLocator(20))

# Plot best 5/worst 5

## Best 5

Time series

In [ ]:
nrows = 5
fig, ax = plt.subplots(
    nrows=nrows,
    figsize=(10,10),
    sharex=True
)

for row_idx in range(nrows):
    score_idx = row_idx
    timeseries = timeseries_df.loc[scores_sorted.index[score_idx]]
    
    ax[row_idx].plot(timeseries)
    ax[row_idx].xaxis.set_major_locator(ticker.MultipleLocator(20))
    ax[row_idx].set_title(f'Rank {score_idx+1}')

fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor="none", top=False, bottom=False, left=False, right=False)
plt.grid(False)
plt.xlabel("Time (min)")
plt.ylabel("Flavin fluorescence, normalised (AU)")

Periodograms

In [ ]:
nrows = 5
fig, ax = plt.subplots(
    nrows=nrows,
    figsize=(5,10),
    sharex=True,
)

for row_idx in range(nrows):
    score_idx = row_idx
    freqs_axis = freqs_dropna.loc[scores_sorted.index[score_idx]]
    power_axis = power_dropna.loc[scores_sorted.index[score_idx]]
     
    ax[row_idx].plot(freqs_axis, power_axis)
    ax[row_idx].set_ylim((0,50))
    ax[row_idx].set_title(f'Rank {score_idx+1}')

fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor="none", top=False, bottom=False, left=False, right=False)
plt.grid(False)
plt.xlabel(r"Frequency ($\mathrm{min}^{-1}$)")
plt.ylabel("Power (dimensionless)")

## Worst 5

Time series

In [ ]:
nrows = 5
fig, ax = plt.subplots(
    nrows=nrows,
    figsize=(10,10),
    sharex=True,
)

for row_idx in range(nrows):
    score_idx = len(scores_sorted) - 1 - row_idx
    timeseries = timeseries_df.loc[scores_sorted.index[score_idx]]
    
    ax[row_idx].plot(timeseries)
    ax[row_idx].xaxis.set_major_locator(ticker.MultipleLocator(20))
    ax[row_idx].set_title(f'Rank {score_idx+1}')

fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor="none", top=False, bottom=False, left=False, right=False)
plt.grid(False)
plt.xlabel("Time (min)")
plt.ylabel("Flavin fluorescence, normalised (AU)")

Periodograms

In [ ]:
nrows = 5
fig, ax = plt.subplots(
    nrows=nrows,
    figsize=(5,10),
    sharex=True,
)

for row_idx in range(nrows):
    score_idx = len(scores_sorted) - 1 - row_idx
    freqs_axis = freqs_dropna.loc[scores_sorted.index[score_idx]]
    power_axis = power_dropna.loc[scores_sorted.index[score_idx]]
     
    ax[row_idx].plot(freqs_axis, power_axis)
    ax[row_idx].set_ylim((0,50))
    ax[row_idx].set_title(f'Rank {score_idx+1}')

fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor="none", top=False, bottom=False, left=False, right=False)
plt.grid(False)
plt.xlabel(r"Frequency ($\mathrm{min}^{-1}$)")
plt.ylabel("Power (dimensionless)")

# ROC

Number of independent frequencies ($M$)

(Not a for loop because it's the same for each time series, at least for the data I use)

In [ ]:
time_axis = timeseries_dropna.columns.to_list()
time_axis = [float(time) for time in time_axis]

In [ ]:
sampling_pd = time_axis[1] - time_axis[0]
l_ts = time_axis[-1] - time_axis[0]
f_lb = 1/l_ts
f_ub = 0.5 * (1/sampling_pd)
M = f_ub * l_ts

In [ ]:
M

Classifications

In [ ]:
def fdr_classify(scores, q):
    scores_sorted = scores.sort_values(ascending=False)
    n_ts = len(scores_sorted)
    classifications = []
    for k in range(n_ts):
        tmp = -np.log(1 - (1 - ((q*(k+1))/n_ts))**(1/M))
        classification = scores_sorted[k] >= tmp
        classifications.append(classification)
    classifications = np.array(classifications)
    classifications_df = pd.DataFrame(classifications, scores_sorted.index, ['classification'])
    return classifications_df

In [ ]:
q = 1e-8
classifications_df = fdr_classify(scores, q)

In [ ]:
classifications_df

False positives and true positives

In [ ]:
true_labels = labels_df.loc[classifications_df.index].score.to_list()
predicted_labels = classifications_df.classification.to_list()

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(true_labels, predicted_labels)
true_negative, false_positive, false_negative, true_positive = conf_matrix.ravel()
FPR = false_positive / (false_positive + true_negative)
TPR = true_positive / (true_positive + false_negative)

print(f"False Positive Rate: {FPR}")
print(f"True Positive Rate: {TPR}")

Sweep false discovery rates

In [ ]:
#vec = np.power(10, np.linspace(-14, -8, 10))
vec = np.power(10, np.linspace(-8, -1, 100))

In [ ]:
FPR_axis = []
TPR_axis = []

for q in vec:
    classifications_df = fdr_classify(scores, q)
    true_labels = labels_df.loc[classifications_df.index].score.to_list()
    predicted_labels = classifications_df.classification.to_list()
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    true_negative, false_positive, false_negative, true_positive = conf_matrix.ravel()
    FPR = false_positive / (false_positive + true_negative)
    TPR = true_positive / (true_positive + false_negative)
    FPR_axis.append(FPR)
    TPR_axis.append(TPR)

ROC curve

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
ax.plot(FPR_axis, TPR_axis, marker='o')
ax.plot([0,1],[0,1])
ax.set_xlim((0,1))
ax.set_ylim((0,1))
ax.set_xlabel("False positive rate")
ax.set_ylabel("True positive rate")